# 风险管理作业4
-----
2501210025 叶云鹏

In [13]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
import os
from scipy.stats import norm

warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

## （一）.股票投资组合VaR计算

选取具有代表性的十只股票构成股票组合，要求十只股票属于不同的行业。收集这十只股票的历史数据，要求至少两年的数据。根据历史数据，估计日收益率的期望和方差，以及十只股票的日收益率的相关系数。根据这十只股票，设计投资组合，并计算该投资组合的边际VaR和成分VaR

In [14]:
# 第一步：根据原始数据计算日频对数收益率return_df
def get_data(comb_stock_codes):
    
    raw_data_list = []
    for r,d,fs in os.walk('../data/csmar_close'):
        for f in fs:
            if (not ('[DES]' in f)) and ('txt' in f):
                file_path = os.path.join(r,f)
                #print(file_path)
                raw_data_list.append(pd.read_csv(file_path,
                            sep='\t',
                            dtype={'Stkcd': str})[['Trddt','Stkcd','Adjprcwd']])
    working_data  =  pd.concat(raw_data_list,axis=0).set_index(['Trddt','Stkcd'])
    workding_df = working_data.unstack('Stkcd')
    df = workding_df.T.dropna().T
    df.columns = df.columns.get_level_values(1)

    return df[comb_stock_codes]

comb_stock_codes = ['000001','002330','000607','000021','600448',
                    '688233','000901','600722','900948','601208']

price_df = get_data(comb_stock_codes)
price_df.index = pd.to_datetime(price_df.index)
price_df
return_df = np.log(price_df.shift(-1)/price_df)


In [15]:
# 第二步：估计日收益率的期望、方差和相关系数
expected_returns = return_df.mean()
variances = return_df.var()
correlation_matrix = return_df.corr()
covariance_matrix = return_df.cov()  # 后续用于投资组合计算

print("\n日收益率期望：")
display(expected_returns.to_frame(name='期望收益率'))

print("\n日收益率方差：")
display(variances.to_frame(name='方差'))

print("\n日收益率相关系数矩阵：")
display(correlation_matrix)


日收益率期望：


,期望收益率
Stkcd,
000001,-0.000101
002330,-0.000288
000607,0.000049
000021,0.000252
600448,-0.000132
688233,0.000080
000901,0.000349
600722,0.000411
900948,0.001199



日收益率方差：


,方差
Stkcd,
000001,0.000336
002330,0.000647
000607,0.000870
000021,0.000755
600448,0.000650
688233,0.001474
000901,0.000637
600722,0.000898
900948,0.000441



日收益率相关系数矩阵：


Stkcd,000001,002330,000607,000021,600448,688233,000901,600722,900948,601208
Stkcd,,,,,,,,,,
000001,1.000000,0.189848,0.136566,0.140570,0.113534,0.067478,0.137355,0.190699,0.221797,0.125790
002330,0.189848,1.000000,0.378947,0.244086,0.371026,0.214227,0.287330,0.288838,0.151713,0.207565
000607,0.136566,0.378947,1.000000,0.323114,0.386023,0.278561,0.335369,0.284906,0.133918,0.210792
000021,0.140570,0.244086,0.323114,1.000000,0.211220,0.490969,0.411897,0.213016,0.125993,0.379280
600448,0.113534,0.371026,0.386023,0.211220,1.000000,0.249660,0.256545,0.333190,0.154253,0.202154
688233,0.067478,0.214227,0.278561,0.490969,0.249660,1.000000,0.357354,0.228089,0.083645,0.382659
000901,0.137355,0.287330,0.335369,0.411897,0.256545,0.357354,1.000000,0.327389,0.196895,0.357760
600722,0.190699,0.288838,0.284906,0.213016,0.333190,0.228089,0.327389,1.000000,0.270858,0.223439
900948,0.221797,0.151713,0.133918,0.125993,0.154253,0.083645,0.196895,0.270858,1.000000,0.176979


(a)若假设X_t服从正态，记上述方差协方差矩阵为$\Sigma$,均值$\mu$,联合分布为$N(\mu,\Sigma)$

(b) 给出组合 $b = (1/10，1/10，1/10，...,1/10)^{'}$
由
则$W_t \sim N(-V_tb^{'}\mu,{V_t}^2b^{'}\Sigma b)$


In [16]:
# 第三步：设计投资组合
# 简单起见，假设等权重投资组合（每只股票权重 10%）

In [17]:
weights = np.array([0.1] * 10)
portfolio_df = pd.DataFrame({
    '股票': return_df.columns,
    '权重': weights,
    '期望收益率': expected_returns,
    '方差': variances
})
print("\n投资组合设计（等权重）：")
display(portfolio_df)

# 计算投资组合期望收益率和标准差
portfolio_expected_return = np.dot(weights, expected_returns)
portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))
portfolio_std = np.sqrt(portfolio_variance)

print(f"\n投资组合日期望收益率：{portfolio_expected_return:.6f}")
print(f"投资组合日方差：{portfolio_variance:.6f}")
print(f"投资组合日标准差：{portfolio_std:.6f}")


投资组合设计（等权重）：


,股票,权重,期望收益率,方差
Stkcd,,,,
000001,000001,0.1,-0.000101,0.000336
002330,002330,0.1,-0.000288,0.000647
000607,000607,0.1,0.000049,0.000870
000021,000021,0.1,0.000252,0.000755
600448,600448,0.1,-0.000132,0.000650
688233,688233,0.1,0.000080,0.001474
000901,000901,0.1,0.000349,0.000637
600722,600722,0.1,0.000411,0.000898
900948,900948,0.1,0.001199,0.000441



投资组合日期望收益率：0.000271
投资组合日方差：0.000245
投资组合日标准差：0.015639


In [18]:
# 第四步：计算投资组合 VaR（假设正态分布，95% 置信水平）
# VaR 表示在给定时间段内、特定置信水平下可能的最大损失
# 95% 日 VaR，使用 5% 尾部的 z 分数（单尾，负值表示损失）
confidence_level = 0.95
z = norm.ppf(0.05)  # ≈ -1.64485
# 参数 VaR 公式：VaR = mu_p + z * sigma_p（收益率阈值，低于此值的占 5%）
portfolio_var_threshold = portfolio_expected_return + z * portfolio_std
portfolio_var_loss = -portfolio_var_threshold  # 损失值（正值表示损失）

print(f"\n95% 日投资组合 VaR（收益率阈值）：{portfolio_var_threshold:.6f}（低于此收益率的概率为 5%）")
print(f"95% 日投资组合 VaR（损失值）：{portfolio_var_loss:.6f}（最差 5% 的预期最大损失）")


95% 日投资组合 VaR（收益率阈值）：-0.025454（低于此收益率的概率为 5%）
95% 日投资组合 VaR（损失值）：0.025454（最差 5% 的预期最大损失）


In [19]:
# 第五步：计算边际 VaR 和成分 VaR
# 边际 VaR (MVaR)：VaR 对资产权重的偏导数
# 正态假设下：MVaR_i = mu_i + z * (Cov * w)_i / sigma_p
marginal_var = expected_returns + z * (covariance_matrix @ weights) / portfolio_std
# 成分 VaR (CVaR)：权重 * 边际 VaR
component_var = weights * marginal_var
# 成分 VaR 之和应等于投资组合 VaR 阈值

var_df = pd.DataFrame({
    '股票': return_df.columns,
    '权重': weights,
    '边际 VaR': marginal_var,
    '成分 VaR': component_var
})
var_df.loc['总计', '成分 VaR'] = component_var.sum()

print("\n边际 VaR 和成分 VaR（针对 95% VaR 阈值）：")
display(var_df)

print(f"成分 VaR 总和：{component_var.sum():.6f}（应与投资组合 VaR 阈值一致：{portfolio_var_threshold:.6f}）")


边际 VaR 和成分 VaR（针对 95% VaR 阈值）：


,股票,权重,边际 VaR,成分 VaR
Stkcd,,,,
000001,000001,0.1,-0.010518,-0.001052
002330,002330,0.1,-0.024276,-0.002428
000607,000607,0.1,-0.030070,-0.003007
000021,000021,0.1,-0.028819,-0.002882
600448,600448,0.1,-0.024113,-0.002411
688233,688233,0.1,-0.041113,-0.004111
000901,000901,0.1,-0.026456,-0.002646
600722,600722,0.1,-0.028851,-0.002885
900948,900948,0.1,-0.012447,-0.001245


成分 VaR 总和：-0.025454（应与投资组合 VaR 阈值一致：-0.025454）


## 给定投资组合计算VaR